# 1. Import Dependencies


In [41]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [40]:
Discrete(3)

Discrete(3)

In [42]:
Box(0,1,shape=(3,3)).sample()

array([[0.15588076, 0.6246996 , 0.45121133],
       [0.82921636, 0.22961852, 0.80146426],
       [0.16723669, 0.17235762, 0.589155  ]], dtype=float32)

In [43]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[138, 143, 223],
       [ 15, 174, 151],
       [118, 164, 108]])

In [44]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([64.59393], dtype=float32))

In [45]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([59.234367], dtype=float32))])

In [46]:
MultiBinary(4).sample()

array([1, 0, 1, 1], dtype=int8)

In [47]:
MultiDiscrete([5,2,2]).sample()

array([1, 0, 1])

# 3. Building an Environment

In [48]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [49]:
env=ShowerEnv()

In [50]:
env.observation_space.sample()

array([58.4618], dtype=float32)

In [51]:
env.reset()

array([40.])

In [52]:
from stable_baselines3.common.env_checker import check_env

In [53]:
check_env(env, warn=True)

AssertionError: The observation returned by the `reset()` method does not match the given observation space

# 4. Test Environment

In [58]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-38
Episode:2 Score:-54
Episode:3 Score:10
Episode:4 Score:0
Episode:5 Score:-14


# 5. Train Model

In [55]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, verbose=1,)
model.learn(total_timesteps=4000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -22.5    |
| time/              |          |
|    fps             | 868      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -28.2       |
| time/                   |             |
|    fps                  | 615         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010391099 |
|    clip_fraction        | 0.0693      |
|    clip_range           | 0.2         |
|    entropy_loss  

# 6. Save Model

In [38]:
model.save('PPO')

evaluate_policy(model, env, n_eval_episodes=10, )

(59.2, 0.9797958971132712)

stable_baselines3.ppo.ppo.PPO